In [1]:
import numpy
import pandas as pd
import numpy as np
import nltk
from ast import literal_eval
import treetaggerwrapper as ttw
from nltk.stem.porter import *

C:\Users\Simone\anaconda3\lib\site-packages\treetaggerwrapper.py:739: FutureWarning: Possible nested set at position 8
  punct2find_re = re.compile("([^ ])([[" + ALONEMARKS + "])",
C:\Users\Simone\anaconda3\lib\site-packages\treetaggerwrapper.py:2043: FutureWarning: Possible nested set at position 152
  DnsHostMatch_re = re.compile("(" + DnsHost_expression + ")",
C:\Users\Simone\anaconda3\lib\site-packages\treetaggerwrapper.py:2067: FutureWarning: Possible nested set at position 409
  UrlMatch_re = re.compile(UrlMatch_expression, re.VERBOSE | re.IGNORECASE)
C:\Users\Simone\anaconda3\lib\site-packages\treetaggerwrapper.py:2079: FutureWarning: Possible nested set at position 192
  EmailMatch_re = re.compile(EmailMatch_expression, re.VERBOSE | re.IGNORECASE)


In [2]:
recipes = pd.read_csv("data/RAW_recipes.csv")
recipes["steps"] = recipes["steps"].apply(literal_eval)
recipes["ingredients"] = recipes["ingredients"].apply(literal_eval)
recipes["tags"] = recipes["tags"].apply(literal_eval)

In [3]:
recipes["tags"][1]

['30-minutes-or-less',
 'time-to-make',
 'course',
 'main-ingredient',
 'cuisine',
 'preparation',
 'occasion',
 'north-american',
 'breakfast',
 'main-dish',
 'pork',
 'american',
 'oven',
 'easy',
 'kid-friendly',
 'pizza',
 'dietary',
 'northeastern-united-states',
 'meat',
 'equipment']

In [4]:
recipes["steps"][0]

['make a choice and proceed with recipe',
 'depending on size of squash , cut into half or fourths',
 'remove seeds',
 'for spicy squash , drizzle olive oil or melted butter over each cut squash piece',
 'season with mexican seasoning mix ii',
 'for sweet squash , drizzle melted honey , butter , grated piloncillo over each cut squash piece',
 'season with sweet mexican spice mix',
 'bake at 350 degrees , again depending on size , for 40 minutes up to an hour , until a fork can easily pierce the skin',
 'be careful not to burn the squash especially if you opt to use sugar or butter',
 'if you feel more comfortable , cover the squash with aluminum foil the first half hour , give or take , of baking',
 'if desired , season with salt']

In [5]:
recipes["ingredients"][0]

['winter squash',
 'mexican seasoning',
 'mixed spice',
 'honey',
 'butter',
 'olive oil',
 'salt']

In [3]:
preprocessed_sentence = ''
lemma_sentence = ''
def tree_tag(text, tagger):
    tagged_text = tagger.tag_text(text)
    return ttw.make_tags(tagged_text)

In [4]:
def tag_ingredients(ingredients):
    
    tagged_ingredients = []
    for ingredient in ingredients:
    
        sentence = tree_tag(ingredient, tagger)

        for tag in sentence:
        # remove special characters
            if isinstance(tag, ttw.Tag):

                # stem
                stem = stemmer.stem(tag.lemma)
                
                if tag.pos in ['NN0', 'NN1', 'NN2']:
                    tagged_ingredients.append(stem.split('|')[0])
    return tagged_ingredients


In [5]:
def tag_sentence(original_sentence, ingredients):
    
    sentence = tree_tag(original_sentence, tagger)

    step_ingredients = []
    step_activities = []
    
    for tag in sentence:
    # remove special characters
        if isinstance(tag, ttw.Tag):
            
            # stem
            stem = stemmer.stem(tag.lemma)
            if stem in ingredients:
                step_ingredients.append(tag.lemma.split('|')[0])
            if tag.pos in ['VVB','VVZ','VVI',
                           'VVD','VVN', 'VVP','VVG']:#VVG not sure
                step_activities.append(tag.lemma.split('|')[0])
                
    if len(step_activities) == 0:
        # default activity if no verb is present
        step_activities.append('use')

    return step_ingredients, step_activities

            

In [75]:
categories =['cheesecake']
max_recipes = 1000
tagger = ttw.TreeTagger(TAGLANG='en')
stemmer = PorterStemmer()

case_id_col = []
event_id_col = []
order_col = []
activity_col = []
ingredient_col = []

event_cnt = 0 
recipe_cnt = 0

for i in range(0,recipes.shape[0]):

    if len(np.intersect1d(categories,recipes["tags"][i])) == len(categories) :
        #print(recipes["tags"][i])
        recipe_cnt += 1
        if recipe_cnt < max_recipes and recipes["minutes"][i] < 1440:
            ingredients = tag_ingredients(recipes["ingredients"][i])

            minutesPerStep = recipes["minutes"][i]/len(recipes["steps"][i])
            time = 0

            for original_sentence in recipes["steps"][i]:
                step_ingredients, step_activities = tag_sentence(original_sentence,ingredients)

                # synchronous events
                for ingredient in step_ingredients:
                    for activity in step_activities:
                        case_id_col.append(i)
                        event_id_col.append(event_cnt)
                        event_cnt += 1
                        order_col.append(time)

                        activity_col.append(activity)
                        ingredient_col.append(ingredient)

                time += minutesPerStep
            

In [76]:
recipe_log = pd.DataFrame({'case_id': case_id_col,
                           'event_id': event_id_col,
                           'order': order_col,
                           'activity': activity_col,
                           'ingredient': ingredient_col})
recipe_log

,case_id,event_id,order,activity,ingredient
0,68,0,0.000,sprinkle,gelatin
1,68,1,0.000,sprinkle,milk
2,68,2,33.750,add,milk
3,68,3,33.750,dissolve,milk
4,68,4,50.625,add,cream
...,...,...,...,...,...
22980,144045,22980,0.000,fold,chocolate
22981,144045,22981,0.000,garnish,pecan
22982,144045,22982,0.000,sprinkle,pecan
22983,144045,22983,0.000,garnish,chocolate


In [79]:
recipe_log.to_csv('log_' + '_'.join(categories) + '_1000.csv',index = False)

In [77]:
recipe_log['ingredient'].unique()

array(['gelatin', 'milk', 'cream', 'cheese', 'sugar', 'vanilla',
       'chocolate', 'chip', 'crust', 'cooky', 'cookie', 'crumb', 'butter',
       'egg', 'flour', 'almond', 'protein', 'powder', 'cinnamon',
       'nutmeg', 'splenda', 'margarine', 'graham', 'cracker', 'tofu',
       'vegan', 'pudding', 'whip', 'mix', 'blueberry', 'pie', 'yolk',
       'extract', 'wafer', 'cocoa', 'salt', 'butterscotch', 'maple',
       'syrup', 'lemon', 'juice', 'cornstarch', 'zests', 'pumpkin',
       'spice', 'ricotta', 'mascarpone', 'fruit', 'substitute',
       'strawberry', 'schnapps', 'pretzel', 'peanut', 'dough', 'kirsch',
       'cherry', 'maraschinos', 'shaving', 'gingersnaps', 'gingersnap',
       'allspice', 'caramel', 'sauce', 'water', 'white', 'pineapple',
       'oreo', 'coffee', 'amaretto', 'coconut', 'corn', 'spray', 'nut',
       'liqueur', 'jam', 'peach', 'cottage', 'yogurt', 'biscuit', 'curd',
       'oats', 'buttermilk', 'soda', 'whiskey', 'and', 'andes', 'candy',
       'mint', 'gro

In [78]:
recipe_log['case_id'].unique()

array([    68,     96,    149,    158,    190,    262,    279,    410,
          419,    485,    552,    955,    960,   1092,   1100,   1127,
         1234,   1348,   1418,   1431,   1477,   1514,   1597,   1805,
         1809,   1926,   2005,   2014,   2337,   2924,   3256,   3387,
         3626,   3672,   3788,   3974,   4011,   4032,   4048,   4093,
         4103,   4137,   4311,   4409,   4433,   4436,   4437,   4483,
         4518,   4605,   4653,   4758,   4788,   4933,   5087,   5130,
         5243,   5244,   5249,   5796,   6100,   6401,   6446,   6478,
         6530,   6659,   6672,   6696,   6730,   7019,   7331,   7604,
         7705,   7729,   8026,   8237,   8276,  10135,  10306,  10549,
        10790,  10879,  10899,  10903,  10904,  11039,  11801,  13100,
        13116,  13129,  13140,  13155,  13164,  13165,  13722,  14725,
        15469,  15592,  15645,  16324,  16349,  16350,  16470,  16471,
        17008,  17011,  17012,  17013,  17152,  17194,  17211,  17255,
      